In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
airbnb=pd.read_csv('Data-Project.csv',index_col=False)

In [4]:
airbnb.head(2)

,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,description,accommodates,zipcode,neighborhood,total_restaurants_1,total_restaurants_reviews_1,restaurants_avg_rating_1,restaurants_yelp_5,total_restaurants_reviews_5,restaurants_avg_rating_5,total_bars_1,total_bars_reviews_1,bars_avg_rating_1,bars_yelp_5,total_bars_reviews_5,bars_avg_rating_5,nearest_station,distance_to_station,crime_score,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,0.39,1,1,"Beautiful, spacious skylit studio in the heart...",1,10018,Chelsea and Clinton,4.0,964.0,3.875,78.0,40022.0,4.057692,1.0,10.0,4.5,21.0,8701.0,4.000000,42nd St - Bryant Pk,0.093386,3.666079,7,94.0,9.0,9.0,10.0,10.0,10.0,9.0
1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,295,4.67,1,1,Urban retreat: enjoy 500 s.f. floor in 1899 br...,3,11238,Central Brooklyn,NaN,NaN,NaN,27.0,4086.0,4.240741,NaN,NaN,NaN,12.0,790.0,4.208333,Classon Ave,0.417452,3.554833,75,90.0,9.0,9.0,10.0,9.0,10.0,9.0


In [5]:
yelp=pd.read_csv('yelp_cleaned.csv',index_col=False)
yelp.drop(['Unnamed: 0'],axis=1,inplace=True)

In [6]:
yelp=yelp[yelp['borough']!='New Jersey']

In [7]:
categories = []
for iter in yelp['category title']:
    categories.extend(str.split(str(iter), sep=','))

categories = pd.Series(categories)
categories = categories.str.lstrip()
categories.value_counts()

Coffee & Tea              1008
Bars                       535
Italian                    400
Cafes                      343
Cocktail Bars              294
American (New)             280
Mexican                    211
Lounges                    190
Pizza                      187
Bakeries                   163
Pubs                       161
American (Traditional)     146
Seafood                    145
Sports Bars                137
Japanese                   124
Korean                     118
Thai                       118
Chinese                    117
Wine Bars                  104
Dive Bars                  101
Caribbean                   95
Breakfast & Brunch          91
Latin American              88
French                      88
Mediterranean               87
Indian                      78
Burgers                     72
Desserts                    60
Sushi Bars                  58
Greek                       57
                          ... 
Supper Clubs                 1
Weight L

In [8]:
def unpack_categories(df, column):
    for index, value in df[[column]].iterrows():
        df.loc[index, 'is_restaurant'] = ('Bars' not in str(value)) 
        df.loc[index, 'is_bar'] = ('Bars' in str(value)) | ('Nightlife' in str(value))
    return df

In [9]:
yelp_df_copy = yelp.copy()

In [10]:
yelp_df_copy = unpack_categories(yelp_df_copy, 'category title')

In [11]:
yelp_df_copy.is_restaurant.value_counts()

True     6166
False    1372
Name: is_restaurant, dtype: int64

In [12]:
airbnb_nhood=pd.DataFrame(airbnb['neighborhood'].value_counts().reset_index())

In [13]:
airbnb_nhood.isnull().sum()

index           0
neighborhood    0
dtype: int64

In [14]:
yelp_nhood = pd.DataFrame(yelp_df_copy['neighborhood'].value_counts().reset_index())

In [15]:
match_hoods = pd.merge(airbnb_nhood, yelp_nhood,on=['index'], how='outer',suffixes=('_airbnb', '_yelp'))

In [16]:
#Count records where there is no match on each side.
airbnb_nomatch = match_hoods.neighborhood_airbnb[match_hoods.neighborhood_yelp.isnull()]
print('There are ' + str(airbnb_nomatch.sum()) + ' AirBnB listings with no match in the Yelp data.')

yelp_nomatch = match_hoods.neighborhood_yelp[match_hoods.neighborhood_airbnb.isnull()]
print('There are ' + str(yelp_nomatch.sum()) + ' Yelp listings with no match in the AirBnB data.')

There are 824.0 AirBnB listings with no match in the Yelp data.
There are 106.0 Yelp listings with no match in the AirBnB data.


In [17]:
match_hoods=match_hoods.dropna()

In [18]:
yelp_df_copy=yelp_df_copy[~(yelp_df_copy.neighborhood.isin(["Yonkers", "New Rochelle"]))]

In [19]:
#hAVERSINE: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
from math import sin, cos, sqrt, atan2, radians

def compute_distance(lat1, lon1, lat2, lon2):

    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [20]:
yelp_restaurants=yelp_df_copy.loc[yelp_df_copy['is_restaurant'] == True]

In [21]:
yelp_restaurants.drop(['is_bar'],axis=1,inplace=True)

C:\Apps\HarshD\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [22]:
yelp_bar=yelp_df_copy.loc[yelp_df_copy['is_bar'] == True]
yelp_bar.drop(['is_restaurant'],axis=1,inplace=True)

C:\Apps\HarshD\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [24]:
a_hoods = airbnb[['id_airbnb', 'neighborhood', 'latitude', 'longitude']]
a_hoods.head()

,id_airbnb,neighborhood,latitude,longitude
0,2595,Chelsea and Clinton,40.75362,-73.98377
1,3831,Central Brooklyn,40.68514,-73.95976
2,5099,Gramercy Park and Murray Hill,40.74767,-73.97500
3,5121,Central Brooklyn,40.68688,-73.95596
4,5178,Chelsea and Clinton,40.76489,-73.98493


In [25]:
y_hoods_r = yelp_restaurants[['id', 'neighborhood', 'latitude', 'longitude','review_count','rating']]
y_hoods_r.head()

,id,neighborhood,latitude,longitude,review_count,rating
2,fAu6e8uCg1lfqIQ1PKNZyA,Gramercy Park and Murray Hill,40.743772,-73.984627,13,4.5
7,1hScmfYV42XshcprL79MAQ,Gramercy Park and Murray Hill,40.749948,-73.976572,206,4.0
9,mn6JM5lHuSSsG9KSXck3ww,Lower Manhattan,40.704280,-74.009919,754,3.0
11,tiwrAjwddw-3TSrnWSYD7w,Gramercy Park and Murray Hill,40.750580,-73.980180,346,4.0
18,Ea2kgAIGHoVQA8t9wL-RKw,Greenwich Village and Soho,40.719240,-74.000160,73,3.0


In [26]:
y_hoods_b = yelp_bar[['id', 'neighborhood', 'latitude', 'longitude','review_count','rating']]
y_hoods_b.head()

,id,neighborhood,latitude,longitude,review_count,rating
0,33tYNCEBSlDaQOgRq8-L9g,Chelsea and Clinton,40.757920,-73.983750,6,5.0
1,rVwQcrogYw-O2SsAk51iYA,Upper East Side,40.774379,-73.963308,302,4.0
3,JZ06I46tmJlvYA_NZ7zyxw,Chelsea and Clinton,40.754295,-73.989000,559,4.0
4,mvAjuaS1Jp6dsN4a-i_Fyw,Chelsea and Clinton,40.763365,-73.992299,287,4.5
5,GRhA1hH1edG48tj2T5I6CA,Gramercy Park and Murray Hill,40.758074,-73.968724,149,4.0


In [27]:
restaurants = pd.merge(a_hoods, y_hoods_r, how='inner', on='neighborhood', suffixes = ('_airbnb', '_yelp'))
restaurants.shape

(12002757, 9)

In [28]:
restaurants['distance'] = list(map(compute_distance, restaurants['latitude_airbnb'], 
                            restaurants['longitude_airbnb'], restaurants['latitude_yelp'],
                            restaurants['longitude_yelp']))

In [29]:
restaurants_1m = restaurants[restaurants['distance']<=1]
#filter for distance < .5
restaurants_2m =restaurants[restaurants['distance']<=2]
print(restaurants_1m.shape)
print(restaurants_2m.shape)

(4027296, 10)
(8482590, 10)


In [31]:
restaurants_1m

,id_airbnb,neighborhood,latitude_airbnb,longitude_airbnb,id,latitude_yelp,longitude_yelp,review_count,rating,distance
1,2595,Chelsea and Clinton,40.75362,-73.98377,HZoA3-oZiS3DbMonlgclAw,40.760980,-73.977313,38,4.0,0.982932
2,2595,Chelsea and Clinton,40.75362,-73.98377,vZE7WQQZaxQRJ0eOaAAItQ,40.758715,-73.988284,65,4.0,0.682505
4,2595,Chelsea and Clinton,40.75362,-73.98377,ahMxvJI5SnqHUccFSpYWwg,40.750540,-73.990550,116,4.0,0.666138
5,2595,Chelsea and Clinton,40.75362,-73.98377,GxhrrAGhi91hwDRsAVApRA,40.759377,-73.982094,299,4.0,0.655736
8,2595,Chelsea and Clinton,40.75362,-73.98377,OSjrW57ZsA6JOL-x0MmGBQ,40.760806,-73.978302,6,4.0,0.922565
9,2595,Chelsea and Clinton,40.75362,-73.98377,mbkBCEx8mgw9Qroo7xywPg,40.760057,-73.987177,498,4.0,0.771361
10,2595,Chelsea and Clinton,40.75362,-73.98377,sDajjybHSYFw8gTVbknPGA,40.752132,-73.991852,329,4.0,0.700815
13,2595,Chelsea and Clinton,40.75362,-73.98377,DCTWsTYqkbwJ-pFCIwmHew,40.755390,-73.988090,132,4.0,0.413827
14,2595,Chelsea and Clinton,40.75362,-73.98377,NMuZQuK6UU32VlSF6khh0w,40.749863,-73.989868,943,3.5,0.662300
15,2595,Chelsea and Clinton,40.75362,-73.98377,4KqGAtsXe7FXDRvnRQCjVA,40.747237,-73.991702,258,3.5,0.975034


In [32]:
#For everything within .1 mile:
restaurants_1mile = restaurants_1m.groupby(['id_airbnb']).agg({'id':'count',
                                                        'review_count':'sum',
                                                        'rating':'mean'})

In [33]:
restaurants_1mile  = restaurants_1mile.rename(columns={'id':'total_restaurants_1mile', 
                                          'review_count':'total_restaurants_reviews_1mile', 
                                          'rating':'restaurants_avg_rating_1mile'})
restaurants_1mile.head()

,total_restaurants_1mile,total_restaurants_reviews_1mile,restaurants_avg_rating_1mile
id_airbnb,,,
2595,245,145949,4.022449
3831,90,14108,4.205556
5099,161,58927,4.102484
5121,65,8909,4.184615
5178,243,160031,4.051440


In [34]:
#For everything within .1 mile:
restaurants_2mile = restaurants_2m.groupby(['id_airbnb']).agg({'id':'count',
                                                        'review_count':'sum',
                                                        'rating':'mean'})
restaurants_2mile = restaurants_2mile.rename(columns={'id':'restaurants_yelp_2mile', 
                                          'review_count':'total_restaurants_reviews_2mile', 
                                          'rating':'restaurants_avg_rating_2mile'})

In [35]:
all_restaurants = pd.merge(restaurants_1mile , restaurants_2mile , how='right', on='id_airbnb')
all_restaurants.head()

,total_restaurants_1mile,total_restaurants_reviews_1mile,restaurants_avg_rating_1mile,restaurants_yelp_2mile,total_restaurants_reviews_2mile,restaurants_avg_rating_2mile
id_airbnb,,,,,,
2595,245.0,145949.0,4.022449,503,280531,4.045726
3831,90.0,14108.0,4.205556,204,38246,4.117647
5099,161.0,58927.0,4.102484,301,120765,4.043189
5121,65.0,8909.0,4.184615,210,39095,4.114286
5178,243.0,160031.0,4.051440,415,232469,4.036145


In [44]:
bars = pd.merge(a_hoods, y_hoods_b, how='inner', on='neighborhood', suffixes = ('_airbnb', '_yelp'))
#print(bars.shape)
bars['distance'] = list(map(compute_distance, bars['latitude_airbnb'], 
                            bars['longitude_airbnb'], bars['latitude_yelp'],
                            bars['longitude_yelp']))
#print(bars.head())
#filter for distance < 1
bars_1m = bars[bars['distance']<=1]
#filter for distance < 2
bars_2m =bars[bars['distance']<=2]
#print(bars_1.shape)
#print(bars_5.shape)
#For everything within .1 mile:
bars_1mile = bars_1m.groupby(['id_airbnb']).agg({'id':'count',
                                                        'review_count':'sum',
                                                        'rating':'mean'})
bars_1mile.head()
bars_1mile  = bars_1mile.rename(columns={'id':'total_bars_1mile', 
                                          'review_count':'total_bars_reviews_1mile', 
                                          'rating':'bars_avg_rating_1mile'})
#print(bars_1mile.head())
#For everything within .1 mile:
bars_2mile = bars_2m.groupby(['id_airbnb']).agg({'id':'count',
                                                        'review_count':'sum',
                                                        'rating':'mean'})
bars_2mile = bars_2mile.rename(columns={'id':'bars_yelp_2mile', 
                                          'review_count':'total_bars_reviews_2mile', 
                                          'rating':'bars_avg_rating_2mile'})
all_bars = pd.merge(bars_1mile , bars_2mile , how='right', on='id_airbnb')
print(all_bars.head())

           total_bars_1mile  total_bars_reviews_1mile  bars_avg_rating_1mile  \
id_airbnb                                                                      
2595                   57.0                   21609.0               4.035088   
3831                   34.0                    3678.0               4.088235   
5099                   39.0                    9939.0               4.025641   
5121                   23.0                    1407.0               4.173913   
5178                   65.0                   23492.0               4.076923   

           bars_yelp_2mile  total_bars_reviews_2mile  bars_avg_rating_2mile  
id_airbnb                                                                    
2595                   135                     47706               4.007407  
3831                    59                      6714               4.135593  
5099                    70                     14269               4.100000  
5121                    61                      6

In [45]:
rest_df=pd.merge(all_restaurants, all_bars, on='id_airbnb', how='left')

In [46]:
airbnb_df=pd.merge(airbnb, rest_df, on='id_airbnb', how='left')
airbnb_df.head(2)

,id_airbnb,host_id,borough,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,description,accommodates,zipcode,neighborhood,total_restaurants_1,total_restaurants_reviews_1,restaurants_avg_rating_1,restaurants_yelp_5,total_restaurants_reviews_5,restaurants_avg_rating_5,total_bars_1,total_bars_reviews_1,bars_avg_rating_1,bars_yelp_5,total_bars_reviews_5,bars_avg_rating_5,nearest_station,distance_to_station,crime_score,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,total_restaurants_1mile,total_restaurants_reviews_1mile,restaurants_avg_rating_1mile,restaurants_yelp_2mile,total_restaurants_reviews_2mile,restaurants_avg_rating_2mile,total_bars_1mile,total_bars_reviews_1mile,bars_avg_rating_1mile,bars_yelp_2mile,total_bars_reviews_2mile,bars_avg_rating_2mile
0,2595,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,10,48,0.39,1,1,"Beautiful, spacious skylit studio in the heart...",1,10018,Chelsea and Clinton,4.0,964.0,3.875,78.0,40022.0,4.057692,1.0,10.0,4.5,21.0,8701.0,4.000000,42nd St - Bryant Pk,0.093386,3.666079,7,94.0,9.0,9.0,10.0,10.0,10.0,9.0,245.0,145949.0,4.022449,503.0,280531.0,4.045726,57.0,21609.0,4.035088,135.0,47706.0,4.007407
1,3831,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,295,4.67,1,1,Urban retreat: enjoy 500 s.f. floor in 1899 br...,3,11238,Central Brooklyn,NaN,NaN,NaN,27.0,4086.0,4.240741,NaN,NaN,NaN,12.0,790.0,4.208333,Classon Ave,0.417452,3.554833,75,90.0,9.0,9.0,10.0,9.0,10.0,9.0,90.0,14108.0,4.205556,204.0,38246.0,4.117647,34.0,3678.0,4.088235,59.0,6714.0,4.135593


In [47]:
airbnb_df.isnull().sum()

id_airbnb                              0
host_id                                0
borough                                0
neighbourhood_group                    0
latitude                               0
longitude                              0
room_type                              0
price                                  0
minimum_nights                         0
number_of_reviews                      0
reviews_per_month                      0
calculated_host_listings_count         0
availability_365                       0
description                            0
accommodates                           0
zipcode                                0
neighborhood                           0
total_restaurants_1                24795
total_restaurants_reviews_1        24795
restaurants_avg_rating_1           24795
restaurants_yelp_5                  2823
total_restaurants_reviews_5         2823
restaurants_avg_rating_5            2823
total_bars_1                       36531
total_bars_revie

In [48]:
airbnb_df.to_csv('Data-Project.csv')